In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def kernel(x1,x2,l):
    return np.exp(-1.0/l**2*np.sum((np.expand_dims(x1,axis=2) - np.expand_dims(x2,axis=1))**2, axis = 3))

def GP(X,A,x,l):
    k_xX = kernel(x,X,l)
    return np.squeeze(np.matmul(k_xX,  A),axis=(2,))

def normalize(minv, maxv, y):
    return 2*(y-minv)/(maxv-minv)-1.0

In [3]:
def gen_data(n_data, dim, n_gp_samples, n_mc_samples, l):
    X = np.random.uniform(low = -1.0, high = 1.0, size = (n_data, n_gp_samples, dim))
    Y = np.random.uniform(low = -1.0, high = 1.0, size = (n_data, n_gp_samples))

    K_XX = kernel(X,X,l)
    A = np.linalg.solve(K_XX, np.expand_dims(Y,axis=2))
    
    mc_samples = np.random.uniform(low = -1.0, high = 1.0, size = [1, n_mc_samples, dim])
    y = GP(X,A,mc_samples,l)

    min_vals = np.min(y, axis = 1).reshape(n_data,1)
    max_vals = np.max(y, axis = 1).reshape(n_data,1)

    return (X, A, min_vals, max_vals)

In [16]:
# Data
n_data = 10000
n_gp_samples = 6
l = 0.3
n_mc_samples = 1000
dim = 5

params = np.array([n_data, n_gp_samples, n_mc_samples, l, dim])

In [17]:
X, A, minv, maxv = gen_data(n_data, dim, n_gp_samples, n_mc_samples, l)

In [18]:
np.save("./Data/"+str(dim)+"D/params", params)
np.save("./Data/"+str(dim)+"D/X", X)
np.save("./Data/"+str(dim)+"D/A", A)
np.save("./Data/"+str(dim)+"D/minv", minv)
np.save("./Data/"+str(dim)+"D/maxv", maxv)
